In [1]:
!pip install -q TurkishStemmer

In [2]:
import pandas as pd
import numpy as np
import re
from TurkishStemmer import TurkishStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

import joblib
import os
from google.colab import files

In [3]:
# -----------------------  O  ---------------------------- #
             # Veri yukleme ve duzenleme #
# -----------------------  O  ---------------------------- #

In [5]:
train_dosya = "hirsizlikVeriNPSetiTRAIN.csv"
test_dosya = "hirsizlikVeriNPSetiTEST.csv"

df_train = pd.read_csv(train_dosya)
df_test = pd.read_csv(test_dosya)

In [6]:
# -----------------------  O  ---------------------------- #

In [7]:
stemmer = TurkishStemmer()

# Turkcede bulunan baglac gibi kelimeler (Stopwords)
stopwords = set(['ve', 'ile', 'ama', 'fakat', 'lakin', 'ancak', 'çünkü', 'bir', 'bu', 'şu', 'o', 'da', 'de'])

def metin_temizle(text):
    if not isinstance(text, str): return ""

    # Once hepsini kucuk harfe cevirmeliyiz
    text = text.lower()

    # Sonra nokta, bosluk. sayi gibi bilesenleri kaldiriyoruz
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)

    # Kelimelere ayır (Tokenization)
    kelimeler = text.split()

    # Basta belirledigimiz Stopword'lerin temizli ve kok bulma (Stemming)
    temiz_kelimeler = []
    for kelime in kelimeler:
        if kelime not in stopwords:
            try:
                kok = stemmer.stem(kelime)
                temiz_kelimeler.append(kok)
            except:
                temiz_kelimeler.append(kelime) # Hata cikarsa ole birakiyoruz

    # Tekrar birlestiriyoruz
    return " ".join(temiz_kelimeler)

In [8]:
# -----------------------  O  ---------------------------- #

In [9]:
df_train['clean_sanik'] = df_train['sanik_ifadesi'].apply(metin_temizle)
df_train['clean_avukat'] = df_train['avukat_savunmasi'].apply(metin_temizle)

df_test['clean_sanik'] = df_test['sanik_ifadesi'].apply(metin_temizle)
df_test['clean_avukat'] = df_test['avukat_savunmasi'].apply(metin_temizle)

In [10]:
# -----------------------  O  ---------------------------- #

In [11]:
# Ornek durum

print(f"Orjinal metin: {df_train['sanik_ifadesi'].iloc[0]}")
print(f"Temizlenmis metin: {df_train['clean_sanik'].iloc[0]}")

Orjinal metin: Sayın Başkan Beni boşuna tutuyorsunuz. O mal zaten benim hakkımdı, geri aldım. Çıkarın beni buradan.
Temizlenmis metin: say başkan ben boş tutuyor mal zaten ben hakk ger al çıkar ben bura


In [12]:
# -----------------------  O  ---------------------------- #
                    # Vektorizasyon #
# -----------------------  O  ---------------------------- #

In [13]:
# Etiket kodlama (Target Encoding)
le_sanik = LabelEncoder()
le_avukat = LabelEncoder()

# Etiketleri Train verisinden öğreniyoruz
y_sanik_train = le_sanik.fit_transform(df_train['sanikTutumu'])
y_avukat_train = le_avukat.fit_transform(df_train['avukatStratejisi'])

y_sanik_test = le_sanik.transform(df_test['sanikTutumu'])
y_avukat_test = le_avukat.transform(df_test['avukatStratejisi'])

print(f"Sanik siniflari: {le_sanik.classes_}")
print(f"Avukat siniflari: {le_avukat.classes_}")

Sanik siniflari: ['Agresif' 'Celiski' 'Inkar' 'Pisman' 'Sessiz']
Avukat siniflari: ['AkliDenge' 'DelilYetersizligi' 'IndirimOdakli' 'SahsiCezasizlik'
 'VasifDegisikligi']


In [14]:
# TF-IDF Vektorlestirme
tfidf_sanik = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
tfidf_avukat = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))

In [15]:
# Train'e FIT (Öğren), Test'e TRANSFORM (Uygula)
X_sanik_train_vec = tfidf_sanik.fit_transform(df_train['clean_sanik'])
X_avukat_train_vec = tfidf_avukat.fit_transform(df_train['clean_avukat'])

X_sanik_test_vec = tfidf_sanik.transform(df_test['clean_sanik'])
X_avukat_test_vec = tfidf_avukat.transform(df_test['clean_avukat'])

In [16]:
# -----------------------  O  ---------------------------- #
                    # Model kismi #
# -----------------------  O  ---------------------------- #

In [17]:
# Sanik tutumu
model_sanik = LogisticRegression(max_iter=1000)
model_sanik.fit(X_sanik_train_vec, y_sanik_train)

LogisticRegression(max_iter=1000)

In [18]:
# Avukat Stratejisi
model_avukat = LogisticRegression(max_iter=1000)
model_avukat.fit(X_avukat_train_vec, y_avukat_train)

LogisticRegression(max_iter=1000)

In [19]:
# -----------------------  O  ---------------------------- #
                    # Sonuc kismi #
# -----------------------  O  ---------------------------- #

In [20]:
print("Sanik icin sonuclar: ")
y_pred_sanik = model_sanik.predict(X_sanik_test_vec)
print(classification_report(y_sanik_test, y_pred_sanik, target_names=le_sanik.classes_))
print("-" * 50)

print("Avukat icin sonuclar: ")
y_pred_avukat = model_avukat.predict(X_avukat_test_vec)
print(classification_report(y_avukat_test, y_pred_avukat, target_names=le_avukat.classes_))

Sanik icin sonuclar: 
              precision    recall  f1-score   support

     Agresif       1.00      1.00      1.00       418
     Celiski       1.00      1.00      1.00       411
       Inkar       1.00      1.00      1.00       501
      Pisman       1.00      1.00      1.00       474
      Sessiz       1.00      1.00      1.00       196

    accuracy                           1.00      2000
   macro avg       1.00      1.00      1.00      2000
weighted avg       1.00      1.00      1.00      2000

--------------------------------------------------
Avukat icin sonuclar: 
                   precision    recall  f1-score   support

        AkliDenge       1.00      0.97      0.99       196
DelilYetersizligi       1.00      0.99      0.99       954
    IndirimOdakli       0.92      1.00      0.96       350
  SahsiCezasizlik       1.00      0.98      0.99       208
 VasifDegisikligi       1.00      0.97      0.99       292

         accuracy                           0.98      2000


In [21]:
# -----------------------  O  ---------------------------- #
                 # Cikti almak icin #
# -----------------------  O  ---------------------------- #

In [22]:
# Kayıt klasörü oluştur
kayit_klasoru = "egitilmis_modeller"
if not os.path.exists(kayit_klasoru):
    os.makedirs(kayit_klasoru)

# Sanik (Vectorizer + Model + LabelEncoder)
joblib.dump(model_sanik, f'{kayit_klasoru}/model_sanik.pkl')
joblib.dump(tfidf_sanik, f'{kayit_klasoru}/tfidf_sanik.pkl')
joblib.dump(le_sanik, f'{kayit_klasoru}/le_sanik.pkl')

# Avukat
joblib.dump(model_avukat, f'{kayit_klasoru}/model_avukat.pkl')
joblib.dump(tfidf_avukat, f'{kayit_klasoru}/tfidf_avukat.pkl')
joblib.dump(le_avukat, f'{kayit_klasoru}/le_avukat.pkl')

# zip islemi
!zip -r egitilmis_modeller.zip egitilmis_modeller

files.download('egitilmis_modeller.zip')

  adding: egitilmis_modeller/ (stored 0%)
  adding: egitilmis_modeller/le_sanik.pkl (deflated 34%)
  adding: egitilmis_modeller/model_sanik.pkl (deflated 20%)
  adding: egitilmis_modeller/model_avukat.pkl (deflated 45%)
  adding: egitilmis_modeller/le_avukat.pkl (deflated 34%)
  adding: egitilmis_modeller/tfidf_sanik.pkl (deflated 74%)
  adding: egitilmis_modeller/tfidf_avukat.pkl (deflated 69%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>